In [1]:
#Code origin
#
#Author: Alexander Valentini (alexander.valentini@epfl.ch)
#Made on: 26/5-2024
#Made for: Project report for course CS-552. submission on 2/6-2024


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import AutoPeftModelForCausalLM
from datasets import load_dataset
import numpy as np
#Using quantization, but this should probably not be in the final version:
from pathlib import Path
import os

 
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    llm_int8_threshold=6.0,
#    llm_int8_has_fp16_weight=False,
#    bnb_4bit_compute_dtype=torch.bfloat16,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#)

#model_name = 'stabilityai/stablelm-zephyr-3b'
model_name = 'AlexVal/dpo_model'

#safetensors_path = 'sft_stablelm_zephyr_3b/29.05.2024_v4'
#safetensors_path = 'models/dpo_model/merged_model'
#safetensors_path = 'models/dpo_model'
#os.path.abspath(os.getcwd())

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = AutoPeftModelForCausalLM.from_pretrained(
#     safetensors_path,
#     torch_dtype=torch.float16,
#     attn_implementation="sdpa",

#).to(device)

#merged_model = model.merge_and_unload()
#merged_model.save_pretrained('models/dpo_model/merged_model',safe_serialization=True)

tokenizer = AutoTokenizer.from_pretrained("new_tokenizer")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alex\.cache\huggingface\hub\models--AlexVal--dpo_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/610M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [2]:
model

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50278, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affin

In [21]:
#os.path.isdir('sft_stablelm_zephyr_3b_debug')

In [22]:
#model = AutoPeftModelForCausalLM.from_pretrained(
#     'sft_stablelm_zephyr_3b_debug',
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
# )
# # Merge LoRA and base model and save
#merged_model = model.merge_and_unload()

#model = AutoModelForCausalLM.from_pretrained(safetensors_path, from_tf=False, from_flax=False, use_safetensors = True).to(device)
#merged_model.resize_token_embeddings(len(tokenizer))

In [3]:
#model = merged_model
prompt = [{'role': 'user', 'content': 'List 3 synonyms for the word "tiny"'}]
inputs = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    return_tensors='pt'
)

tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.8,
    do_sample=True,
     pad_token_id=tokenizer.pad_token_id
)

print(tokenizer.decode(tokens[0], skip_special_tokens=False))

<|user|>
List 3 synonyms for the word "tiny"<|endoftext|>
<|assistant|>
1. forlorn
2. meagre
3. miniature<|endoftext|>


In [2]:
import huggingface_hub
model.push_to_hub(
 repo_id="dpo_model_new",
 commit_message="Commiting model"
)

adapter_model.safetensors:   0%|          | 0.00/801M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlexVal/dpo_model_new/commit/cb9c3c5e995c328005a745ecb7e898f9f27a6fdb', commit_message='Commiting model', commit_description='', oid='cb9c3c5e995c328005a745ecb7e898f9f27a6fdb', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
tokenizer.push_to_hub(
 repo_id="dpo_model_new",
 commit_message="new_tokenizer"
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alex\.cache\huggingface\hub\models--AlexVal--dpo_model_new. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/AlexVal/dpo_model_new/commit/fbf85089deccb0ef1ccf8ac06c7e6d3d26ae6ef8', commit_message='new_tokenizer', commit_description='', oid='fbf85089deccb0ef1ccf8ac06c7e6d3d26ae6ef8', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
tokenizer.model_max_length

2048